### Imports

In [29]:
import pandas as pd
import numpy as np
import duckdb
import requests
from shapely.geometry import Point
import geopandas as gpd

### Databases:

In [2]:
# BiciMAD Database:

db1 = '../data/bicimad.db'

# Places of Interest Database;

db2 = 'https://datos.madrid.es/egob/catalogo/209426-0-templos-catolicas.json'

### Acquisition Function:

In [3]:
def acquisition(database1, database2):
    db_connect = duckdb.connect(database1)
    bicimad_df1 = duckdb.query('SELECT name AS "BiciMAD Station", address AS "Station Location", \
        "geometry.coordinates" AS "Coordinates" FROM main.bicimad_stations', connection=db_connect).df()
    
    response = requests.get(database2)
    json_data = response.json()
    places_df1 = pd.json_normalize(json_data['@graph'])
    
    return bicimad_df1, places_df1

In [4]:
bicimad_df1, places_df1 = acquisition(db1, db2)

### Wrangling Functions:

In [5]:
# BiciMAD Wrangling Function:

def wrangling_bicimad(df):
    df[['Longitude', 'Latitude']] = df['Coordinates'].str.strip('[]').str.split(pat = ',',\
                                                    expand = True, regex=False).apply(pd.to_numeric)
    df = df.drop('Coordinates', axis=1).reindex(columns=['BiciMAD Station','Station Location','Latitude','Longitude'])
    return df

In [6]:
bicimad_df = wrangling_bicimad(bicimad_df1)

In [7]:
# Places Wrangling Function:

def wrangling_places(df):
    df = df[['title','@type','address.street-address','location.latitude','location.longitude']]
    df = df.rename(columns={'title': 'Place of Interest','@type':'Type of Place',\
                                        'address.street-address':'Place Address',\
                                       'location.latitude':'Latitude','location.longitude':'Longitude'})
    df['Type of Place'] = df['Type of Place'].str.extract('/entidadesYorganismos/(\w+)')
    df['Type of Place'] = df['Type of Place'].str.findall('[A-Z][^A-Z]*').str.join(' ')
    df['Place Address'] = df['Place Address'].str.title()
    df=df.loc[df['Type of Place'] == "Templos Iglesias Catolicas"]

    return df

In [8]:
places_df = wrangling_places(places_df1)

### Analysis Functions:

In [9]:
# Function to combine both Dataframes using .merge
def combine_df(df1, df2):
    combined_df = df2.merge(df1, how='cross')
    return combined_df

In [10]:
combined_df = combine_df(places_df, bicimad_df)

In [11]:
# transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
def to_mercator(lat, long):
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

# Function to return the distance in metres between to latitude/longitude pair point in degrees
def distance_meters(lat_start, long_start, lat_finish, long_finish):
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [54]:
# Function to get the distance from two points and add it in a new column

def distance_df(combined_df):
    lat_x = np.array(combined_df['Latitude_x'])
    lon_x = np.array(combined_df['Longitude_x'])
    lat_y = np.array(combined_df['Latitude_y'])
    lon_y = np.array(combined_df['Longitude_y'])

    # Aplica la función "distance_meters" a los arrays NumPy de coordenadas
    dist = np.vectorize(distance_meters)(lat_x, lon_x, lat_y, lon_y)
    
    # Asigna el resultado a la columna "Distance" de tu DataFrame
    combined_df['Distance'] = dist
    
    return combined_df

In [56]:
%%time

df_distance = distance_df(combined_df)

CPU times: user 16min 17s, sys: 52.8 s, total: 17min 10s
Wall time: 17min 17s


In [25]:
# Function to filter the dataframe with the distances
def final_df(df_distance):
    final_df = combined_df.loc[combined_df.groupby("Place of Interest")["Distance"].idxmin()]
    final_df = final_df.drop(['Latitude_x','Longitude_x','Latitude_y','Longitude_y'], axis=1)
    final_df = final_df.reset_index(drop=True)
    final_df['Distance'] = final_df['Distance'].round(0)
    return final_df

In [26]:
df_final = final_df(df_distance)

In [27]:
# Function to create a .csv from the final dataframe
def create_csv(df_final):
        csv_df = df_final[['Place of Interest', 'Type of Place','Place Address','BiciMAD Station','Station Location']]
        csv_df.to_csv('Main Challenge.csv', index=False)
        return csv_df

In [28]:
csv = create_csv(df_final)
csv

,Place of Interest,Type of Place,Place Address,BiciMAD Station,Station Location
0,Basílica Jesús de Medinaceli,Templos Iglesias Catolicas,Plaza Jesus 2,27 - Huertas,Calle Jesús nº 1
1,Basílica La Milagrosa,Templos Iglesias Catolicas,Calle Garcia De Paredes 45,125 - General Martínez Campos,Calle Fernández de la Hoz nº 29
2,Capilla Nuestra Señora de la Bien Aparecida,Templos Iglesias Catolicas,Calle Pio Baroja 2,71 - Doce de Octubre,Calle Doce de Octubre nº 28
3,Capilla de San Ignacio de los Vascos,Templos Iglesias Catolicas,Calle Principe 31,52 - Plaza de Santa Ana,Plaza de Santa Ana nº 10
4,Capilla de la Virgen de Nuria,Templos Iglesias Catolicas,Calle Juan De Urbieta 5,178 - Junta Municipal Retiro,Calle León Gil de Palacio nº 3
...,...,...,...,...,...
352,Santuario Nuestra Señora de Schönstatt,Templos Iglesias Catolicas,Calle Serrano 97,250 - Serrano 113,Calle Serrano nº 113B
353,Santuario Nuestra Señora de los Peligros,Templos Iglesias Catolicas,Calle Joaquin Costa 49,140 - Velázquez 130,Calle Velázquez nº 130
354,Segundo Monasterio de la Visitación (Las Salesas),Templos Iglesias Catolicas,Calle San Bernardo 72,12 - San Hermenegildo,Calle San Bernardo nº 85
355,Templo Eucarístico de San Martín de Tours,Templos Iglesias Catolicas,Calle Desenga&Amp;Ntilde;O 26,2 - Miguel Moya,Calle Miguel Moya nº 1
